In [ ]:
!pip install -U kaleido

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from urllib.request import Request, urlopen
import json
import plotly.io as pio

In [ ]:
url_usos = "https://cdn.buenosaires.gob.ar/datosabiertos/datasets/secretaria-de-desarrollo-urbano/relevamiento-usos-suelo/relevamiento-usos-del-suelo-2017.csv"

In [ ]:
usos = pd.read_csv(url_usos)

In [ ]:
len(usos)

561722

In [ ]:
usos.head()

,X,Y,SMP,CALLE,NUM,TIPO1_16,TIPO2_16,PISOS_16,NOMBRE,OBSERVACIO,BARRIO,COMUNA,5_DIG,4_DIG,3_DIG,2_DIG,RAMA,SUBRAMA,SSRAMA
0,-58.455634,-34.596891,047-040A-001C,MONTENEGRO,124,E,VIVIENDA,1,NaN,NaN,CHACARITA,15.0,3,0,0,0,EDIFICIOS,VIVIENDA,NaN
1,-58.455699,-34.596945,047-040A-002,NUEVA ZELANDIA,4820,EP,METALURGIA,1,RODOCA S.A.,NaN,CHACARITA,15.0,28900,2890,289,28,"FABRICACIÓN DE PRODUCTOS ELABORADOS DE METAL, ...",FABRICACIÓN DE PRODUCTOS ELABORADOS DE METAL NCP,NaN
2,-58.455750,-34.597013,047-040A-003,NUEVA ZELANDIA,4830,E,CASA,1,NaN,E X GAP,CHACARITA,15.0,4,0,0,0,EDIFICIOS,CASA,NaN
3,-58.455810,-34.597074,047-040A-004,NUEVA ZELANDIA,4840,GAP,GARAGE PRIVADO,1,NaN,NaN,CHACARITA,15.0,63312,6331,633,63,SERVICIOS COMPLEMENTARIOS AL TRANSPORTE,SERVICIOS COMPLEMENTARIOS AL TRANSPORTE TERRESTRE,"ESTACIONAMIENTOS, COCHERAS Y GARAJES"
4,-58.455879,-34.597129,047-040A-005,NUEVA ZELANDIA,4850,GAP,GARAGE PRIVADO,1,NaN,NaN,CHACARITA,15.0,63312,6331,633,63,SERVICIOS COMPLEMENTARIOS AL TRANSPORTE,SERVICIOS COMPLEMENTARIOS AL TRANSPORTE TERRESTRE,"ESTACIONAMIENTOS, COCHERAS Y GARAJES"


In [ ]:
parrillas = usos[usos["TIPO2_16"] == "PARRILLA"]

In [ ]:
parrillas["direc"] = parrillas["CALLE"] + " " + parrillas["NUM"].astype(str)

<ipython-input-21-34a7583d181b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  parrillas["direc"] = parrillas["CALLE"] + " " + parrillas["NUM"].astype(str)


In [ ]:
parrillas_CABA = go.Scattergeo(
    lat=parrillas["Y"],
    lon=parrillas["X"],
    mode='markers',
    text=parrillas["direc"],
    marker=dict(color='orange', size=10),
    opacity=1,
    showlegend=False,
    hoverinfo='text'
)

In [ ]:
url_barrios = urlopen('https://cdn.buenosaires.gob.ar/datosabiertos/datasets/barrios/barrios.geojson')

In [ ]:
def coordinates_extract(geojson, empty_list, empty_dict):
  for feature in geojson['features']:
    if feature['geometry']['type'] == "MultiPolygon":
      for polyg in feature['geometry']['coordinates']:
        empty_list.extend(polyg[0])
        empty_list.append([None, None])

In [ ]:
barrios = json.load(url_barrios)

barrios_coords = []

coordinates_extract(barrios, barrios_coords, None)

barrios_lons, barrios_lats = zip(*barrios_coords)

In [ ]:
barrios_CABA = go.Scattergeo(
    lat=barrios_lats,
    lon=barrios_lons,
    mode='lines',
    line=dict(color='white', width=1),
    opacity=0.5,
    showlegend=False,
    hoverinfo='skip'
)

In [ ]:
def maps_layout_and_annotation(fig):
  fig.update_layout(
      paper_bgcolor='rgba(0,0,0,1)',
      margin=dict(
          t = 150,
          b = 0,
          l = 0,
          r = 0
      ),
      height = 900,
      width = 900,
      title = dict(
        text = "<b>Parrillas en Buenos Aires</b><br><sup>Relevamiento de usos del suelo: parrillas.<br><sup><b>Fuente</b>: BA Data.</sup></sup>",
        xanchor = 'left',
        x = 0.0325,
        yanchor = 'top',
        y = 0.95,
        font = dict(
            size = 26,
            family="Courier New",
            color = 'white'
        )
        ),
      geo = dict(
        center = dict(
            lat = -34.616,
            lon = -58.435
        ),
        bgcolor = 'rgba(0,0,0,1)',
        projection_type = 'mercator',
        projection_scale = 1640,
        resolution = 50,
        showlakes = False,
        showcoastlines = False,
        showland = False,
        showocean = False,
        showframe = False,
        showcountries = False,
    )
  )

In [ ]:
mapa_parrillas = go.Figure()

In [ ]:
mapa_parrillas.add_trace(parrillas_CABA)

mapa_parrillas.add_trace(barrios_CABA)

maps_layout_and_annotation(mapa_parrillas)

In [ ]:
mapa_parrillas

In [ ]:
pio.write_html(mapa_parrillas,
               file='mapa_parrillas.html',
               full_html=True)